In [6]:
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, BatchNormalization, ReLU, Flatten, Dense, Add, Dropout
import tensorflow as tf
from keras.datasets import cifar10

In [7]:
#データのロード
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

#０〜１へ正則化
X_train = X_train.astype("float32")/255.0
X_test = X_test.astype("float32")/255.0

# one-hotエンコーディング
Y_train = keras.utils.to_categorical(y_train)
Y_test = keras.utils.to_categorical(y_test)

input_shape = X_train.shape[1:]
train_samples = X_train.shape[0]
test_samples = X_test.shape[0]

# ハイパパラメータ
epochs = 200
batch_size = 50